In [1]:
import tensorflow as tf
import os

graph_def = tf.compat.v1.GraphDef()
labels = []

# These are set to the default names from exported models, update as needed.
filename = "model.pb"
labels_filename = "labels.txt"

# Import the TF graph
with tf.io.gfile.GFile(filename, 'rb') as f:
    graph_def.ParseFromString(f.read())
    tf.import_graph_def(graph_def, name='')

# Create a list of labels.
with open(labels_filename, 'rt') as lf:
    for l in lf:
        labels.append(l.strip())

In [2]:
from PIL import Image
import numpy as np
import cv2

def convert_to_opencv(image):
    # RGB -> BGR conversion is performed as well.
    image = image.convert('RGB')
    r,g,b = np.array(image).T
    opencv_image = np.array([b,g,r]).transpose()
    return opencv_image

def crop_center(img,cropx,cropy):
    h, w = img.shape[:2]
    startx = w//2-(cropx//2)
    starty = h//2-(cropy//2)
    return img[starty:starty+cropy, startx:startx+cropx]

def resize_down_to_1600_max_dim(image):
    h, w = image.shape[:2]
    if (h < 1600 and w < 1600):
        return image

    new_size = (1600 * w // h, 1600) if (h > w) else (1600, 1600 * h // w)
    return cv2.resize(image, new_size, interpolation = cv2.INTER_LINEAR)

def resize_to_256_square(image):
    h, w = image.shape[:2]
    return cv2.resize(image, (256, 256), interpolation = cv2.INTER_LINEAR)

def update_orientation(image):
    exif_orientation_tag = 0x0112
    if hasattr(image, '_getexif'):
        exif = image._getexif()
        if (exif != None and exif_orientation_tag in exif):
            orientation = exif.get(exif_orientation_tag, 1)
            # orientation is 1 based, shift to zero based and flip/transpose based on 0-based values
            orientation -= 1
            if orientation >= 4:
                image = image.transpose(Image.TRANSPOSE)
            if orientation == 2 or orientation == 3 or orientation == 6 or orientation == 7:
                image = image.transpose(Image.FLIP_TOP_BOTTOM)
            if orientation == 1 or orientation == 2 or orientation == 5 or orientation == 6:
                image = image.transpose(Image.FLIP_LEFT_RIGHT)
    return image

In [3]:
#統計資料夾中有幾張圖片
path = 'C:/Users/User/0612_Merge_Json/Test_Image_Cut' #要統計的資料夾
count = len([name for name in os.listdir(path) if os.path.isfile(os.path.join(path, name))])
print(len([name for name in os.listdir(path) if os.path.isfile(os.path.join(path, name))]))

5


In [4]:
#把資料夾中的所有圖片經過此模型後存成一個個json檔
for filename in os.listdir(path):
    # Load from a file
    #imageFile = "C:/Users/User/0601匯出分類器TensorFlow/Test_Image/NET_x_.jpg"
    #imageFile = imageFile.replace('x',str(pic))
    #image = Image.open(imageFile)
    image = Image.open(path+"/"+filename)

    # Update orientation based on EXIF tags, if the file has orientation info.
    image = update_orientation(image)

    # Convert to OpenCV format
    image = convert_to_opencv(image)

    # If the image has either w or h greater than 1600 we resize it down respecting
    # aspect ratio such that the largest dimension is 1600
    image = resize_down_to_1600_max_dim(image)

    # We next get the largest center square
    h, w = image.shape[:2]
    min_dim = min(w,h)
    max_square_image = crop_center(image, min_dim, min_dim)

    # Resize that square down to 256x256
    augmented_image = resize_to_256_square(max_square_image)

    # Get the input size of the model
    with tf.compat.v1.Session() as sess:
        input_tensor_shape = sess.graph.get_tensor_by_name('Placeholder:0').shape.as_list()
        network_input_size = input_tensor_shape[1]

    # Crop the center for the specified network_input_Size
    augmented_image = crop_center(augmented_image, network_input_size, network_input_size)

    # These names are part of the model and cannot be changed.
    output_layer = 'loss:0'
    input_node = 'Placeholder:0'

    with tf.compat.v1.Session() as sess:
        try:
            prob_tensor = sess.graph.get_tensor_by_name(output_layer)
            predictions = sess.run(prob_tensor, {input_node: [augmented_image] })
        except KeyError:
            print ("Couldn't find classification output layer: " + output_layer + ".")
            print ("Verify this a model exported from an Object Detection project.")
            exit(-1)
        
    # Print the highest probability label
    highest_probability_index = np.argmax(predictions)
    print('Classified as: ' + labels[highest_probability_index])
    print()

    # Or you can print out all of the results mapping labels to probabilities.
    label_index = 0

    f1 = open(path+"/"+filename.split('.')[0]+".json", "w")#w代表覆蓋寫入
    f1.writelines(str("{'Classification':"+"'"+labels[highest_probability_index]+"'").replace("'",'"')+",")
    f1.close()
 
    f1 = open(path+"/"+filename.split('.')[0]+".json", "a")#a代表附加寫入
 
    for p in predictions:
        truncated_probablity = np.float64(np.round(p,8))
        print (labels[label_index], truncated_probablity,truncated_probablity[highest_probability_index])
        f1.writelines(str("'Highest_probability':"+str(truncated_probablity[highest_probability_index])).replace("'",'"')+"}")
        label_index += 1
    f1.close()
    print()
  

Classified as: floral

cam [0.00107455 0.01025146 0.55913562 0.03080762 0.0195463  0.04891068] 0.5591356158256531

Classified as: stripe

cam [0.00136816 0.01925036 0.08720058 0.01834706 0.07342443 0.65589035] 0.6558903455734253

Classified as: dot

cam [0.00887337 0.2576853  0.01602122 0.03677806 0.01478669 0.06166568] 0.2576853036880493

Classified as: floral

cam [6.15600002e-05 5.55477012e-03 9.48638260e-01 4.23398986e-03
 1.21450005e-03 8.49816017e-03] 0.9486382603645325

Classified as: floral

cam [7.89000023e-06 1.80304004e-03 8.54296744e-01 2.14958325e-01
 8.22737999e-03 2.63342708e-01] 0.8542967438697815

